Online Webstore Dataset Analysis

In this notebook, you will find commands to read in 3 datasets that help an online webstore track customer sales behaviour.

The 3 main datasets are:

Customer: A dataset containing basic customer information <br>
Product: A dataset containing information about specific products and their prices <br>
Sales: A dataset containing information about purchases made by a customer <br>





In [1]:
from pyspark.sql.types import StructType, StructField, IntegerType, ArrayType, StringType, DateType
from pyspark.sql import SparkSession

# Initialize a SparkSession
spark = (
    SparkSession.builder.appName("Spark Example").getOrCreate()
)

25/06/11 12:53:38 WARN Utils: Your hostname, codespaces-22a5ca resolves to a loopback address: 127.0.0.1; using 10.0.12.164 instead (on interface eth0)
25/06/11 12:53:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/11 12:53:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:

# Define the schema
sales_schema = StructType([
    StructField("SaleID", IntegerType(), True),
    StructField("CustomerID", IntegerType(), True),
    StructField("ProductIDs", ArrayType(IntegerType()), True),
    StructField("Purchase_Date", StringType(), True)  # Using StringType for simplicity
])


sales = spark.read.parquet('sales_dataset.parquet', schema =sales_schema,header=True)
sales.show()

<h5> Question 1: Based on how we have read in the schema for the sales dataset, can you apply a schema to the customer and sales dataset? </h5>

In [2]:
customer = spark.read.parquet('customer_dataset.parquet', header=True)

In [3]:
customer.show()

+----------+-----------+----------+------------+--------------------+
|CustomerId|       Name|       DoB|Phone Number|               Email|
+----------+-----------+----------+------------+--------------------+
|       251|Customer251|1972-07-07| 35452398098|customer251@examp...|
|       252|Customer252|1983-11-16| 53870960356|customer252@examp...|
|       253|Customer253|1991-05-24| 04955017858|customer253@examp...|
|       254|Customer254|1983-10-04| 59412216040|customer254@examp...|
|       255|Customer255|1984-07-12| 00412449253|customer255@examp...|
|       256|Customer256|1979-03-19| 40953435482|customer256@examp...|
|       257|Customer257|1990-05-18| 01745444544|customer257@examp...|
|       258|Customer258|1975-08-22| 14587131559|customer258@examp...|
|       259|Customer259|1971-02-02| 43213131663|customer259@examp...|
|       260|Customer260|1980-02-25| 78948060570|customer260@examp...|
|       261|Customer261|1995-07-24| 28556492385|customer261@examp...|
|       262|Customer

In [4]:
product = spark.read.parquet('product_dataset.parquet', header=True)

In [5]:
product.show()

+---------+---------+-----+
|ProductID|     Item| cost|
+---------+---------+-----+
|       51|Product51|96.34|
|       52|Product52|32.98|
|       53|Product53|25.57|
|       54|Product54|50.34|
|       55|Product55|60.44|
|       56|Product56|56.05|
|       57|Product57|60.02|
|       58|Product58|20.39|
|       59|Product59|86.19|
|       60|Product60|70.27|
|       61|Product61|92.14|
|       62|Product62|59.97|
|       63|Product63|94.48|
|       64|Product64|17.87|
|       65|Product65|63.52|
|       66|Product66|97.33|
|       67|Product67|98.65|
|       68|Product68|13.61|
|       69|Product69|44.98|
|       70|Product70|11.55|
+---------+---------+-----+
only showing top 20 rows



In [13]:
# Register the DataFrame as a SQL temporary view
customer.createOrReplaceTempView("tmp_customer")
sales.createOrReplaceTempView("tmp_sales")
product.createOrReplaceTempView("tmp_product")

<h5> Question 2: Calculate the number of customers that have signed up to the website that have made a purchase on the website </h5>


In [ ]:
spark.sql("""select count(distinct c.customerid) from tmp_customer """).show()

NameError: name 'spark' is not defined

<h5> Question 3: we want to find all the customers that haven't made a purchase yet as part of a marketing campaign, please write some code to generate that - Hint: Make use of the explode function </h5>

In [80]:

#Here we have intentionally picked the first item in the array to give the wrong value, we are instead expecting them to try and explode this array and get the correct answer this way
spark.sql('select sum(cost) from tmp_product as p join tmp_sales as s on element_at(s.productIDs, 1) = p.productID group by p.productID').show()


+------------------+
|         sum(cost)|
+------------------+
|225.35000000000002|
| 661.8600000000001|
|             474.5|
|           1050.48|
|            165.84|
| 463.4699999999999|
|            686.48|
|            369.24|
|349.71999999999997|
| 706.6800000000001|
|             263.5|
|           1180.92|
|443.34000000000003|
| 94.32000000000001|
|            476.64|
|              94.4|
|1645.4299999999998|
| 918.9000000000001|
|            104.37|
|            758.88|
+------------------+
only showing top 20 rows



<h5> Question 4: Please Generate the date that a customer made their second purchase on the website. Hint: Make use of the Row_Number function </h5>

In [28]:
spark.sql('select CustomerID, FIRST_VALUE(Purchase_Date) over (Partition By CustomerID Order By Purchase_Date Desc) as FirstPurchase from tmp_sales').show()


+----------+-------------+
|CustomerID|FirstPurchase|
+----------+-------------+
|       001|   2023-10-23|
|       001|   2023-10-23|
|       001|   2023-10-23|
|       001|   2023-10-23|
|       002|   2023-11-16|
|       002|   2023-11-16|
|       003|   2023-10-15|
|       003|   2023-10-15|
|       003|   2023-10-15|
|       003|   2023-10-15|
|       004|   2023-08-12|
|       004|   2023-08-12|
|       004|   2023-08-12|
|       005|   2023-12-30|
|       005|   2023-12-30|
|       005|   2023-12-30|
|       005|   2023-12-30|
|       007|   2023-08-11|
|       007|   2023-08-11|
|       007|   2023-08-11|
+----------+-------------+
only showing top 20 rows

